In [8]:
#%%% JSON content script
import pandas as pd
import numpy as np
import json
from sqlalchemy import create_engine

connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dmaxreportsrvr.database.windows.net%3B' \
                    'Database%3Dmax_report_db%3B' \
                    'UID%3Dreportadmin%3B' \
                    'PWD%3D#DAff!%nz8r7'\

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT 
            MXRITEMNAME, 
            SALESORDERNUMBER, 
            CUSTOMSDOCUMENTDATE,
            SUM(ORDEREDSALESQUANTITY) AS TotalSalesQuantity
        FROM 
            [dbo].[SalesOrderLineV2Staging]
        WHERE 
            DATAAREAID = 'MNHQ'
            AND (MXRITEMNAME = 'Tegel' OR MXRITEMNAME = 'Vloerverwarming')
        GROUP BY 
            MXRITEMNAME, SALESORDERNUMBER
         """

# Get the email data as a Pandas DataFrame
data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()


In [20]:
# Step 1: Identify duplicated SALESORDERNUMBERs
duplicated_order_numbers = data.pivot_table(index=['SALESORDERNUMBER'], aggfunc='size')[lambda x: x >= 2].index
filtered_data = data[(data['SALESORDERNUMBER'].isin(duplicated_order_numbers))]

item_counts = filtered_data.pivot_table(index='SALESORDERNUMBER', columns='MXRITEMNAME', values='TotalSalesQuantity', aggfunc='sum', fill_value=0)

In [26]:
import seaborn as sns

In [23]:
item_counts

MXRITEMNAME,Tegel,Vloerverwarming
SALESORDERNUMBER,,
MNHQSO0400545,21,1
MNHQSO0401376,30,1
MNHQSO0401578,13,1
MNHQSO0401712,21,1
MNHQSO0402258,33,1
...,...,...
MNHQSO0449764,19,1
MNHQSO0450250,21,1
MNHQSO0450783,25,1


In [25]:
np.unique(item_counts['Tegel'])

array([ 1,  2,  3,  5,  6,  7,  8,  9, 10, 12, 13, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 41, 42, 43, 45, 46, 48, 49, 50, 52, 57, 60, 63, 73, 76],
      dtype=int64)

In [14]:
duplicates = data.pivot_table(index = ['SALESORDERNUMBER'], aggfunc ='size')[lambda x: x >= 2].index

In [15]:
duplicates

Index(['MNHQSO0400545', 'MNHQSO0401376', 'MNHQSO0401578', 'MNHQSO0401712',
       'MNHQSO0402258', 'MNHQSO0402483', 'MNHQSO0405183', 'MNHQSO0405835',
       'MNHQSO0406070', 'MNHQSO0406702',
       ...
       'MNHQSO0448467', 'MNHQSO0448610', 'MNHQSO0448656', 'MNHQSO0448781',
       'MNHQSO0448999', 'MNHQSO0449764', 'MNHQSO0450250', 'MNHQSO0450783',
       'MNHQSO0451034', 'MNHQSO0451176'],
      dtype='object', name='SALESORDERNUMBER', length=150)